In [3]:
import pandas as pd
import xml.etree.ElementTree as ET
from pandas import json_normalize
import json
import requests
from bs4 import BeautifulSoup


In [4]:
# Assuming the JSON content is stored in a file named "simple.json"
simple_json = pd.read_json("simple.json")

# Display the dataframe
simple_json

,id,brand,model,year,owned_by,acquired,price
0,1,Chrysler,Voyager,2001,Wolff-Trantow,28/12/2016,11565
1,2,Volkswagen,Jetta,1995,"Schuppe, Pfeffer and Klein",20/4/2016,52431
2,3,Porsche,Cayenne,2005,Mante Group,11/6/2020,75552
3,4,Porsche,928,1994,Wisozk Group,17/6/2019,30331
4,5,Mercedes-Benz,SL-Class,2007,Schiller-Littel,20/7/2018,62385
...,...,...,...,...,...,...,...
95,96,Dodge,Ram 1500 Club,2000,Schneider-Donnelly,28/9/2016,56256
96,97,GMC,Yukon XL 2500,2001,"Powlowski, Lemke and Wiza",19/4/2016,50163
97,98,Chevrolet,Sportvan G20,1992,"O'Keefe, Rogahn and Gleichner",1/6/2019,8577
98,99,Mitsubishi,3000GT,1993,Hoppe-Kunde,29/12/2018,23180


In [5]:
# Assuming the JSON content is stored in a file named "simple.json"
simple_xml = pd.read_xml("simple.xml", xpath='/car')

# Display the dataframe
simple_xml


,id,brand,model,year,owned_by,acquired,price
0,1,Oldsmobile,Aurora,2003,"Herzog, Rodriguez and Howe",21/10/2016,32571


In [6]:
# Fetch data from the website
response = requests.get("https://edu.frostbit.fi/api/events/en/")
data = response.json()

# Extract variables from data
names = [item.get('name', '') for item in data]  # Fill missing names with empty string
dates = [item.get('date', '') for item in data]  # Fill missing dates with empty string
categories = [', '.join(item['categories']) if item.get('categories') else '' for item in data]
street_addresses = [item['address'].get('street_address', '') for item in data]  # Fill missing street addresses with empty string
postal_codes = [item['address'].get('postal_code', '') for item in data]  # Fill missing postal codes with empty string

# Create a DataFrame
API_data = pd.DataFrame({
    'Name': names,
    'Date': dates,
    'Categories': categories,
    'Street Address': street_addresses,
    'Postal Code': postal_codes
})

# Transpose the DataFrame
API_data_transposed = API_data.T

# Save the DataFrame as a JSON file in vertical format
try:
    API_data_transposed.to_json("Exercise5_API.json", orient="index")
    print("JSON file saved successfully.")
except Exception as e:
    print(f"Error occurred while saving JSON file: {e}")

JSON file saved successfully.


In [7]:
# Load API_DATA.json and safe dataframe into a descriptive variable
API_df = pd.read_json("Exercise5_API.json")
API_df

,Name,Date,Categories,Street Address,Postal Code
0,Playground Loru’s schedule 13.5.-17.5.2024,2024-05-15,"fine arts, music, playing (children's games)",Töölönlahdenkatu 4,100
1,Playground Loru’s schedule 13.5.-17.5.2024,2024-05-17,"fine arts, music, playing (children's games)",Töölönlahdenkatu 4,100
2,Playground Loru’s schedule 13.5.-17.5.2024,2024-05-13,,Töölönlahdenkatu 4,100
3,Playground Loru’s schedule 13.5.-17.5.2024,2024-05-13,"fine arts, music, playing (children's games)",Töölönlahdenkatu 4,100
4,Playground Loru’s schedule 13.5.-17.5.2024,2024-05-16,"fine arts, music, playing (children's games)",Töölönlahdenkatu 4,100
5,Playground Loru’s schedule 13.5.-17.5.2024,2024-05-14,"fine arts, music, playing (children's games)",Töölönlahdenkatu 4,100


In [8]:
# define the URL of the webpage and download the raw HTML
url = "https://en.wikipedia.org/wiki/Rovaniemi"
page = requests.get(url)

# create a "soup object" of the raw web-page
soup = BeautifulSoup(page.content, "html.parser")

coordinates_element = soup.find("span", class_="geo")
coordinates = coordinates_element.get_text()

nickname_element = soup.find("div", class_="ib-settlement-nickname nickname")
nickname = nickname_element.get_text()

print(f"Coordinates of Rovaniemi: {coordinates}")
print(f"Nicknames of Rovaniemi: {nickname}")


Coordinates of Rovaniemi: 66.500; 25.733
Nicknames of Rovaniemi: Arctic Capital; Hometown of Santa Claus


In [9]:
# Define the URL of the webpage and download the raw HTML
url = "https://en.wikipedia.org/wiki/Rovaniemi"
page = requests.get(url)

# Create a "soup object" of the raw web-page
soup = BeautifulSoup(page.content, "html.parser")

# Get the coordinates of Rovaniemi
coordinates_element = soup.find("span", class_="geo")
coordinates = coordinates_element.get_text()

# Get the nickname of Rovaniemi
nickname_element = soup.find("div", class_="ib-settlement-nickname nickname")
nickname = nickname_element.get_text()

# Find and extract the total area of Rovaniemi
infobox_rows = soup.find_all("tr", class_="mergedrow")
population = None
for row in infobox_rows:
    row_text = row.text
    rows = row_text.split("\n")
    for r in rows:
        if "Total" in r:
            if "km" not in r:
                total_population = ''.join(filter(str.isdigit, r))  # Remove non-digit characters from the area
                break
            
# Search for the row containing the total area information
total_area = None
for row in infobox_rows:
    if "Total" in row.text:
        # Extract the total area value from the row
        total_area = row.find("td").text.strip()
        break

# Print the total area
if total_area:
    print("Total Area:", total_area)
else:
    print("Total area not found.")


# Print the results
print(f"Coordinates of Rovaniemi: {coordinates}")
print(f"Nicknames of Rovaniemi: {nickname}")
print(f"Population of Rovaniemi: {total_population}")

Total Area: 8,016.75 km2 (3,095.28 sq mi)
Coordinates of Rovaniemi: 66.500; 25.733
Nicknames of Rovaniemi: Arctic Capital; Hometown of Santa Claus
Population of Rovaniemi: 65286


In [10]:
# URL of List_of_countries_by_average_yearly_temperature
url = "https://en.wikipedia.org/wiki/List_of_countries_by_average_yearly_temperature"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table with class 'wikitable sortable'
table = soup.find('table', class_='wikitable sortable')

In [12]:
data = []
for row in table.find_all('tr')[1:]:
    columns = row.find_all('td')
    country = columns[1].text.strip()
    avg_temp = columns[2].text.strip()
    data.append({'Country': country, 'avg_temp': avg_temp})

# Create a DataFrame for the list of dictionaries
df = pd.DataFrame(data)

# Display dataframe
df

,Country,avg_temp
0,Burkina Faso,29.26 °C (84.67 °F)
1,Mali,29.21 °C (84.58 °F)
2,Aruba,29.17 °C (84.51 °F)
3,Senegal,28.90 °C (84.02 °F)
4,Mauritania,28.82 °C (83.88 °F)
...,...,...
230,Mongolia,1.07 °C (33.93 °F)
231,Russia,−3.79 °C (25.18 °F)
232,Canada,−4.03 °C (24.75 °F)
233,Svalbard and Jan Mayen,−6.78 °C (19.80 °F)


In [14]:
# Safe "https://en.wikipedia.org/wiki/List_of_countries_by_employment_rate" and "https://en.wikipedia.org/wiki/List_of_countries_by_unemployment_rate" urls into their own variables
employment_url = "https://en.wikipedia.org/wiki/List_of_countries_by_employment_rate"
unemployment_url = "https://en.wikipedia.org/wiki/List_of_countries_by_unemployment_rate"

# Define function to scrape tables from urls above, return df(dataframe)
def scrape_table(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", {"class": "wikitable"})
    df = pd.read_html(str(table))[0]
    return df

# Get employment and unemployment data
employment_df = scrape_table(employment_url)
unemployment_df = scrape_table(unemployment_url)

# Display employment_df dataframe
employment_df

C:\Users\Archoste\AppData\Local\Temp\ipykernel_25104\208120442.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Archoste\AppData\Local\Temp\ipykernel_25104\208120442.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


,Country,Rate (%),Ages,Year,Source
0,Iceland,81.9,15–64,2021,OECD[5]
1,Netherlands,80.8,15–64,2021,OECD[5]
2,Switzerland,80.1,15–64,2021,OECD[5]
3,New Zealand *,79.2,15–64,2021,OECD[5]
4,Japan,78.0,15–64,2021,OECD[5]
5,Norway,77.5,15–64,2021,OECD[5]
6,Germany,76.8,15–64,2021,OECD[5]
7,Denmark *,76.5,15–64,2021,OECD[5]
8,Sweden *,76.0,15–64,2021,OECD[5]
9,United Kingdom *,75.5,15–64,2021,OECD[5]


In [15]:
# Display unemployment_df
unemployment_df

,Country,Unemployment rate (%),Source/date of information
0,Afghanistan *,13.3,[4] 2021
1,Albania *,11.7,[5] Q1 2022
2,Algeria *,12.7,[4] 2021
3,American Samoa *,18.0,[6] 2012
4,Andorra *,3.7,[7] 2016
...,...,...,...
218,U.S. Virgin Islands *,13.3,[4] 2021
219,Wallis and Futuna *,8.8,[7] 2013
220,Yemen *,13.6,[4] 2021
221,Zambia *,13.0,[4] 2021


In [17]:
# Merge the employment_df and unemployment_df into one dataframe called combined_df
combined_df = pd.merge(employment_df, unemployment_df, on="Country")

# Display the combined dataframe
combined_df


,Country,Rate (%),Ages,Year,Source,Unemployment rate (%),Source/date of information
0,New Zealand *,79.2,15–64,2021,OECD[5],3.30,[5] Q2 2022
1,Denmark *,76.5,15–64,2021,OECD[5],4.50,[11] June 2022
2,Sweden *,76.0,15–64,2021,OECD[5],7.70,[11] June 2022
3,United Kingdom *,75.5,15–64,2021,OECD[5],3.50,[53] October 2022
4,Australia *,75.3,15–64,2021,OECD[5],3.40,[5] July 2022
5,China *,75.1,15–64,2010,OECD[1],5.40,[5] Q2 2022
6,Estonia *,75.0,15–64,2021,OECD[5],5.70,[11] June 2022
7,Canada *,74.7,15–64,2021,OECD[5],5.80,[15] November 2023
8,Hungary *,73.9,15–64,2021,OECD[5],3.00,[11] June 2022
9,Malta *,73.8,15–64,2020,OECD[1],3.00,[11] June 2022


In [19]:
# Read coffee_sales.csv and temperatures_fixed.csv
coffee_sales_df = pd.read_csv("coffee_sales.csv")
temperatures_df = pd.read_csv("temperatures_fixed.csv")

# Merge the DataFrames coffee_sales_df and temperatures_df
merged_df = pd.merge(coffee_sales_df, temperatures_df, on="Date")
merged_df

,Date,Sales,Target_sales,Total_expenses,Temperature (C)
0,1.10.2012,122,90,76,19.822917
1,1.10.2012,123,90,45,19.822917
2,2.10.2012,107,90,36,19.020602
3,3.10.2012,94,100,21,15.820139
4,4.10.2012,182,80,54,15.110648
...,...,...,...,...,...
1057,23.8.2015,534,360,94,18.253009
1058,24.8.2015,236,160,60,21.114583
1059,25.8.2015,33,0,145,22.575000
1060,26.8.2015,341,230,65,20.204398


In [20]:
# Read the CSV file "temperatures_unfixed.csv" into a DataFrame
temperatures_unfixed_df = pd.read_csv("temperatures_unfixed.csv")

# Drop the columns "Summary" and "Hour" from the DataFrame
temperatures_unfixed_df = temperatures_unfixed_df.drop(["Summary", "Hour"], axis=1)

# Group the DataFrame by the "Date" column and calculate the mean temperature for each date
grouped_temperatures_df = temperatures_unfixed_df.groupby("Date").mean()

# Reset the index of the grouped DataFrame 
grouped_temperatures_df.reset_index(inplace=True)

In [27]:
# Display grouped_temperatures_df
grouped_temperatures_df

,Date,Temperature (C)
0,1.1.2006,4.075000
1,1.1.2007,3.806713
2,1.1.2008,-5.663194
3,1.1.2009,-4.850926
4,1.1.2010,7.807407
...,...,...
4014,9.9.2012,21.729398
4015,9.9.2013,16.082870
4016,9.9.2014,20.099074
4017,9.9.2015,13.582870


In [23]:
# Display coffee sales_df
coffee_sales_df

,Date,Sales,Target_sales,Total_expenses
0,1.10.2012,122,90,76
1,1.10.2012,123,90,45
2,2.10.2012,107,90,36
3,3.10.2012,94,100,21
4,4.10.2012,182,80,54
...,...,...,...,...
1057,23.8.2015,534,360,94
1058,24.8.2015,236,160,60
1059,25.8.2015,33,0,145
1060,26.8.2015,341,230,65


In [24]:
# Display temperatures_df
temperatures_df

,Date,Temperature (C)
0,1.1.2006,4.075000
1,1.1.2007,3.806713
2,1.1.2008,-5.663194
3,1.1.2009,-4.850926
4,1.1.2010,7.807407
...,...,...
4014,9.9.2012,21.729398
4015,9.9.2013,16.082870
4016,9.9.2014,20.099074
4017,9.9.2015,13.582870


In [26]:
# Merge the coffee sales data with the grouped temperature data based on Date
merged_df = pd.merge(coffee_sales_df, grouped_temperatures_df, on="Date")

# Display the merged DataFrame
merged_df

,Date,Sales,Target_sales,Total_expenses,Temperature (C)
0,1.10.2012,122,90,76,19.822917
1,1.10.2012,123,90,45,19.822917
2,2.10.2012,107,90,36,19.020602
3,3.10.2012,94,100,21,15.820139
4,4.10.2012,182,80,54,15.110648
...,...,...,...,...,...
1057,23.8.2015,534,360,94,18.253009
1058,24.8.2015,236,160,60,21.114583
1059,25.8.2015,33,0,145,22.575000
1060,26.8.2015,341,230,65,20.204398


In [29]:
# Load the JSON data from the file
with open("complex.json", "r") as file:
    data = json.load(file)

# Flatten the nested JSON structure into a DataFrame
complex_df = pd.json_normalize(data)

# Display complex_df
complex_df


,id,brand,model,year,price,ownership.company,ownership.acquired,ownership.payment_info.credit_card,ownership.payment_info.iban
0,1,Ford,F250,1997,72213,Johns and Sons,24/10/2017,jcb,SA47 315V KYTA 8FVT NCGN MM7G
1,2,Dodge,Grand Caravan,1999,42370,"Crist, Hyatt and Leannon",7/6/2020,instapayment,FR74 8157 1767 53I2 VRAN ZMIB U11
2,3,Audi,A5,2011,19191,Bosco and Sons,23/11/2018,jcb,LB32 4989 VCVZ S5BZ X2CT JKG3 QRAW
3,4,Hyundai,Veracruz,2012,21956,Haley Inc,5/6/2016,bankcard,BE62 4961 0596 5774
4,5,Chevrolet,TrailBlazer,2009,53066,"Zulauf, Nolan and Franecki",14/1/2021,jcb,FR11 1768 5506 10IU QVPN XU2H O88
...,...,...,...,...,...,...,...,...,...
95,96,Hyundai,Elantra,1997,38400,Keebler and Sons,26/6/2019,visa-electron,RS34 1571 2547 8662 4201 18
96,97,Mercury,Grand Marquis,1996,27284,Wunsch-Ernser,9/11/2019,mastercard,AD85 4944 9836 X0YY JZAF NIXH
97,98,GMC,Safari,1996,17723,Ondricka-Schuster,17/7/2016,jcb,FR22 8203 4892 75GI 8YFM NAXR B47
98,99,Dodge,Ram Van 1500,2000,32892,Heathcote-Gleichner,21/4/2016,jcb,SA43 49ZA FH7V 20XN BYDW ROEX


In [31]:
# Load the JSON data from the file
with open("simple.json", "r") as file:
    data = json.load(file)

# Create variables to store the total_price and amount of the variables
total_price = 0
count = 0
for item in data:
    total_price += item["price"]
    count += 1
average_price = total_price / count
print(f"Average Price: {average_price}$")

Average Price: 42373.86$


In [35]:
import xml.etree.ElementTree as ET
import pandas as pd

# Parse the XML file
tree = ET.parse("complex.xml")
root = tree.getroot()

# Extract data from XML and create a DataFrame
data = []
for car in root.findall("car"):
    car_data = {
        "id": car.find("id").text,
        "brand": car.find("brand").text,
        "model": car.find("model").text,
        "year": car.find("year").text,
        "owned_by": car.find("owned_by").text if car.find("owned_by") is not None else None,
        "acquired": car.find("acquired").text if car.find("acquired") is not None else None,
        "price": car.find("price").text if car.find("price") is not None else None
    }
    data.append(car_data)

# Create dataframe from the data above
complex_xml_df = pd.DataFrame(data)

# Display the complex_xml_df
complex_xml_df


,id,brand,model,year,owned_by,acquired,price
0,1,Mazda,B-Series Plus,1994,None,None,69221
1,2,Ford,F350,2006,None,None,13859
2,3,BMW,7 Series,2012,None,None,78125
3,4,Subaru,Baja,2004,None,None,32941
4,5,Audi,S6,2007,None,None,46761
...,...,...,...,...,...,...,...
95,96,Lexus,IS,2004,None,None,79409
96,97,Oldsmobile,Bravada,2001,None,None,61308
97,98,Dodge,Ram 2500 Club,1996,None,None,12101
98,99,Jeep,Wrangler,2007,None,None,74912
